In [8]:
import random
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from dataclasses import dataclass

In [9]:
seed = 42

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed);

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device {device}')

Using device cuda


### Prepare the data

In [70]:
class ImpurityDataset(Dataset):
    
    def __init__(self, dataframe, fixed_features, labels, feature_scaler=None, label_scaler=None, device=None):
        assert len(labels) % 2 == 0
        
        self.fixed_features = fixed_features
        self.labels = labels
        self.n_samples = len(dataframe)
        
        self.output_length = 2
        self.input_length = 2
        self.sequence_length = len(fixed_features) + (len(labels) // 2) - 1
        
        df_features = dataframe[fixed_features]
        df_labels = dataframe[labels]

        if feature_scaler is not None and label_scaler is not None:
            xs = feature_scaler.transform(df_features.values)
            ys = label_scaler.transform(df_labels.values)
        else:
            xs = df_features.values
            ys = df_labels.values

        feature_data = np.zeros((self.n_samples, self.sequence_length, self.input_length))
        label_data = np.zeros((self.n_samples, self.sequence_length, self.output_length))
        
        for i in range(self.n_samples):
            xi = xs[i]
            xj = ys[i][:-self.output_length]
            
            features = np.zeros((self.sequence_length, self.input_length))

            features[:len(xi), 0] = xi
            features[len(xi):, :] = xj.reshape(-1, self.input_length)
            
            y = np.zeros((self.sequence_length, self.input_length))
            y[len(xi)-1:] = ys[i].reshape(-1, self.output_length)
            
            feature_data[i, :, :] = features
            label_data[i, :, :] = y

        self.feature_data = torch.tensor(feature_data, dtype=torch.float).to(device)
        self.label_data = torch.tensor(label_data, dtype=torch.float).to(device)

    def __len__(self):
        return self.n_samples

    def __getitem__(self, idx):
        return self.feature_data[idx], self.label_data[idx]

In [174]:
def compute_scalers(dataframe, fixed_features, labels, test_size, random_state=None):
    train_df, _ = train_test_split(dataframe, test_size=test_size, random_state=random_state)
    df_features = train_df[fixed_features]
    df_labels = train_df[labels]
    
    feature_scaler = StandardScaler()
    feature_scaler.fit(df_features.values)
    
    label_scaler = StandardScaler()
    label_scaler.fit(df_labels.values)

    return feature_scaler, label_scaler      

In [461]:
file_path = '../data/20230825_144318_10k_EVDoubExp-TExp-wmax5-sparse-hyb_with_perturbation.csv'

#fixed_features = ['beta', 'U', 'Eimp', 'E1', 'E2', 'E3', 'V1', 'V2', 'V3']
fixed_features = ['beta', 'E1', 'E2', 'E3', 'V1', 'V2', 'V3']
labels = ['ReSf1', 'ImSf1', 'ReSf3', 'ImSf3', 'ReSf5', 'ImSf5', 'ReSf7', 'ImSf7', 'ReSf9', 'ImSf9', 'ReSf11', 'ImSf11', 'ReSf13', 'ImSf13', 'ReSf15', 'ImSf15', 'ReSf17', 'ImSf17', 'ReSf19', 'ImSf19', 'ReSf21', 'ImSf21', 'ReSf23', 'ImSf23', 'ReSf25', 'ImSf25', 'ReSf29', 'ImSf29', 'ReSf33', 'ImSf33', 'ReSf37', 'ImSf37', 'ReSf43', 'ImSf43', 'ReSf49', 'ImSf49', 'ReSf57', 'ImSf57', 'ReSf69', 'ImSf69', 'ReSf83', 'ImSf83', 'ReSf101', 'ImSf101', 'ReSf127', 'ImSf127', 'ReSf165', 'ImSf165', 'ReSf237', 'ImSf237', 'ReSf399', 'ImSf399', 'ReSf1207', 'ImSf1207']

df = pd.read_csv(file_path, skiprows=4) # we skip the first four lines, because they are just metadata
df = df[fixed_features + labels]

# remove one special row, looks very weird; ReSf1 = 2.377167465976437e-06
df = df[df['ReSf1'] >= 1e-05]

validation_size = 0.1 # 90% training, 10% for validation

use_scaling = True

if use_scaling:
    feature_scaler, label_scaler = compute_scalers(df, fixed_features, labels, validation_size, seed) # make sure we use the same seed, otherwise the two splits differ!
    dataset = ImpurityDataset(df, fixed_features, labels, feature_scaler, label_scaler, device=device)
else:
    dataset = ImpurityDataset(df, fixed_features, labels, device=device)

indices = list(range(len(dataset)))
train_indices, val_indices = train_test_split(indices, test_size=validation_size, random_state=seed)  # make sure we use the same seed, otherwise the two splits differ!

train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=0)

In [462]:
dataset.__getitem__(0);

In [463]:
sorted(df['ReSf1'].abs())[:10];

### Define the model

In [464]:
class PositionalEncodingL(nn.Module):
    def __init__(self, T, C, dropout):
        super(PositionalEncodingL, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.positional_embedding = nn.Parameter(torch.zeros(T, C))
    
    def forward(self, x):
        B, T, C = x.shape
        position_encoded = self.positional_embedding[:T, :].unsqueeze(0).expand(B, -1, -1)
        x = x + position_encoded
        return self.dropout(x)

In [465]:
@dataclass
class ModelConfig:
    unmasked_input: int
    input_dim: int
    output_dim: int
    sequence_length: int
    
    d_model: int
    nhead: int
    num_layers: int
    dim_feedforward: int
    
    dropout: float
    activation: str
    bias: bool

class AutoregressiveTransformer(nn.Module):
    
    def __init__(self, config, device):
        super(AutoregressiveTransformer, self).__init__()

        self.config = config
        self.input_projection = nn.Linear(config.input_dim, config.d_model)
        self.positional_encoding = PositionalEncodingL(config.sequence_length, config.d_model, config.dropout)
        
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=config.d_model, 
            nhead=config.nhead, 
            dim_feedforward=config.dim_feedforward, 
            dropout=config.dropout,
            activation=config.activation, 
            batch_first=True, 
            norm_first=True, 
            bias=config.bias
        )
        
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=config.num_layers)
        self.output_layer = nn.Linear(config.d_model, config.output_dim)
        self.att_mask = {}
        
    def forward(self, x):        
        x = self.input_projection(x)        
        x = self.positional_encoding(x)

        seq_len = x.size(1)
        if seq_len not in self.att_mask:
            self.att_mask[seq_len] = self.generate_mask(seq_len, device)
        
        output = self.transformer_decoder(x, x, tgt_mask=self.att_mask[seq_len])
        output = self.output_layer(output)
        
        return output

    def generate_mask(self, sequence_length, device):
        mask = torch.full((sequence_length, sequence_length), float('-inf'), device=device)
        mask[:, :self.config.unmasked_input] = 0 
                
        for i in range(self.config.unmasked_input, sequence_length):
            mask[i:, i] = torch.tensor([0] * (sequence_length - i), device=device)
            
        return mask

### Initialize the model

In [706]:
config = ModelConfig(
    unmasked_input = len(fixed_features),
    input_dim = 2,
    output_dim = 2,
    sequence_length = len(fixed_features) + (len(labels) // 2) - 1,
    
    d_model = 64,
    nhead = 4,
    num_layers = 4,
    dim_feedforward = 64 * 4,
    
    dropout = 0.1,
    activation = 'gelu',
    bias = True
)

model = AutoregressiveTransformer(config, device).to(device)
print(sum(p.numel() for p in model.parameters())/1e3, 'k parameters')
criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

269.442 k parameters


In [707]:
def train(model, masked_output, train_loader, optimizer, criterion, device):
    model.train()
    losses = []
    
    for inputs, targets in train_loader:
        
        optimizer.zero_grad()

        outputs = model(inputs)
        outputs[:, :masked_output, :] = torch.zeros((masked_output, outputs.size(2)), device=device)
        loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())

    return np.average(losses)

In [708]:
def validate(model, masked_output, val_loader, criterion, device):
    model.eval()
    losses = []
    
    with torch.no_grad():
        
        for inputs, targets in val_loader:
            
            outputs = model(inputs)
            outputs[:, :masked_output, :] = torch.zeros((masked_output, outputs.size(2)), device=device)
            loss = criterion(outputs, targets)
            losses.append(loss.item())
            
    return np.average(losses)

In [709]:
def validate_mape(model, masked_output, loader, device, use_scaling=use_scaling, epsilon=1e-8):
    model.eval()
    losses = []
    
    with torch.no_grad():
        
        for inputs, targets in loader:
            ogtgt = targets
            outputs = model(inputs)
            outputs[:, :masked_output, :] = torch.zeros((masked_output, outputs.size(2)), device=device)
            
            if use_scaling:
                outputs = torch.tensor(reverse_tranform_output(outputs, masked_output, label_scaler))
                targets = torch.tensor(reverse_tranform_output(targets, masked_output, label_scaler))
            
            ape = torch.abs((targets - outputs) / (targets + epsilon))
            mape = torch.mean(ape) * 100
            
            losses.append(mape.item())

    return np.average(losses)

In [710]:
def reverse_tranform_output(data, masked_output, scaler):
    data = data[:, masked_output:, :]
    B, T, C = data.shape
    data = data.view(B, T*C)
    data = data.cpu().numpy()
    return scaler.inverse_transform(data)

In [711]:
masked_output = len(fixed_features) - 1

validate_mape(model, masked_output, train_loader, device, use_scaling=use_scaling)

109.59513825007848

### Train the model

In [712]:
num_epochs = 200
masked_output = len(fixed_features) - 1

for epoch in range(num_epochs):
    train(model, masked_output, train_loader, optimizer, criterion, device)
    
    train_loss = validate(model, masked_output, train_loader, criterion, device)
    val_loss = validate(model, masked_output, val_loader, criterion, device)
    
    train_mape = validate_mape(model, masked_output, train_loader, device, use_scaling=use_scaling)
    val_mape = validate_mape(model, masked_output, val_loader, device, use_scaling=use_scaling)

    if epoch % 5 == 0:
        n_samples = 100
        sequence_length = 27
        fixed_features_len = len(fixed_features)
        loader = val_loader
        
        reg_error = autoregressive_error(model, loader, n_samples, sequence_length, fixed_features_len, use_scaling)
        print(f"Epoch {(epoch+1):3d}: Train Loss: {train_loss:.6f}, Train MAPE: {train_mape:.3f}%, Val Loss: {val_loss:.6f}, Val MAPE: {val_mape:.3f}%, REG {reg_error:.3f}%")
    else:
        print(f"Epoch {(epoch+1):3d}: Train Loss: {train_loss:.6f}, Train MAPE: {train_mape:.3f}%, Val Loss: {val_loss:.6f}, Val MAPE: {val_mape:.3f}%")
        

Epoch   1: Train Loss: 0.022386, Train MAPE: 25.680%, Val Loss: 0.021374, Val MAPE: 29.629%, REG 119.019%
Epoch   2: Train Loss: 0.019915, Train MAPE: 24.843%, Val Loss: 0.018657, Val MAPE: 29.360%
Epoch   3: Train Loss: 0.018500, Train MAPE: 22.755%, Val Loss: 0.017257, Val MAPE: 27.746%
Epoch   4: Train Loss: 0.017511, Train MAPE: 20.284%, Val Loss: 0.016330, Val MAPE: 25.132%
Epoch   5: Train Loss: 0.016462, Train MAPE: 19.947%, Val Loss: 0.016525, Val MAPE: 23.890%
Epoch   6: Train Loss: 0.014944, Train MAPE: 19.647%, Val Loss: 0.014376, Val MAPE: 23.476%, REG 127.537%
Epoch   7: Train Loss: 0.014167, Train MAPE: 18.709%, Val Loss: 0.014221, Val MAPE: 22.695%
Epoch   8: Train Loss: 0.012831, Train MAPE: 18.766%, Val Loss: 0.013227, Val MAPE: 22.313%
Epoch   9: Train Loss: 0.011581, Train MAPE: 18.846%, Val Loss: 0.012882, Val MAPE: 22.600%
Epoch  10: Train Loss: 0.009797, Train MAPE: 17.338%, Val Loss: 0.010271, Val MAPE: 21.175%
Epoch  11: Train Loss: 0.008593, Train MAPE: 16.990%


KeyboardInterrupt



### Sample from the model

In [669]:
def sample_from_model(model, input, sequence_length, fixed_features_len, device):
    model.eval()

    input = input[:fixed_features_len, :]
    T, C = input.shape
    input = input.view(1, T, C)
    outputs = torch.zeros(sequence_length, 2, device=device)
    
    with torch.no_grad():
    
        for i in range(0, sequence_length):
            output = model(input)
            predictions = output[:, -1, :].view(1, 1, 2)
            outputs[i, :] = predictions
            
            if i == sequence_length:
                break

            input = torch.cat((input, predictions), dim=1)

    return outputs

In [675]:
def calculate_mape(true_values, predictions, epsilon=1e-8):
    mape = torch.mean(torch.abs((true_values - predictions) / (true_values + epsilon))) * 100
    return mape.item()

In [692]:
def autoregressive_error(model, loader, n_samples, sequence_length, fixed_features_len, use_scaling):

    mapes = []
    
    for idx in range(n_samples):
        input = loader.dataset[idx][0]
        
        outputs = sample_from_model(model, input, sequence_length, fixed_features_len, device)
        targets = loader.dataset[idx][1][fixed_features_len-1:, :]

        T, C = outputs.shape
        outputs = outputs.view(1, T, C)
        targets = targets.view(1, T, C)
        
        if use_scaling:
            outputs = torch.tensor(reverse_tranform_output(outputs, 0, label_scaler))
            targets = torch.tensor(reverse_tranform_output(targets, 0, label_scaler))
        
        mapes.append(calculate_mape(targets, outputs))

    return np.mean(mapes)

In [713]:
n_samples = 100
sequence_length = 27
fixed_features_len = len(fixed_features)
loader = train_loader

autoregressive_error(model, loader, n_samples, sequence_length, fixed_features_len, use_scaling)

99.46531008720397

In [ ]:
n_samples = 1000
sequence_length = 27
fixed_features_len = len(fixed_features)
max_features = fixed_features_len + len(labels) - 2

total_mapes = []

for j in range(sequence_length):
    fixed_labels_len = j
    
    mapes = []
    
    for idx in range(n_samples):
        input = val_loader.dataset[idx][0][fixed_labels_len]
        output = sample_from_model(model, input, sequence_length, fixed_features_len, fixed_labels_len, max_features, device)
        
        target = convert(val_loader.dataset[idx][1].reshape(-1))
        output = convert(output.reshape(-1))
        
        mapes.append(calculate_mape(target, output))
    
        if (idx+1) % 1000 == 0:
            print(f"sequence {idx/1000} done")

    total_mapes.append(np.mean(mapes))

print(total_mapes)